# PRI Project, 2nd Delivery

This implementation is the second part of our PRI project. We are Group 14, composed by: 

- Ana Evans nº 86379
- Artur Guimaraes nº 86389
- Francisco Rosa nº 86417


This Notebook showcases the functional part of the second delivery. In each section we  present the tested function and a set of outputs. After each funtion we will mention the structure and the meaning of each input and output. Alternatively, you can include a standard function signature.


In [32]:
# --------------------------------
# 2nd Delivery of the PRI project
# 86379 - Ana Evans
# 86389 - Artur Guimarães
# 86417 - Francisco Rosa
# --------------------------------
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os, os.path
import pickle
import re
import scipy.sparse
import shutil
import sklearn
import spacy
import sys
import time
import whoosh

from textblob import TextBlob
from bs4 import BeautifulSoup
from copy import deepcopy
from heapq import nlargest
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from rank_bm25 import BM25Okapi
from scipy import stats

from whoosh import index
from whoosh import scoring
from whoosh.fields import *
from whoosh.qparser import *
from whoosh.qparser import OrGroup
from whoosh.qparser import QueryParser

from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import *
from sklearn.metrics.cluster import *
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import normalize
from sklearn.svm import LinearSVC


# --------------------------------------------------------------------------------
# ______                  ___________   _____           _                 
# \ ___ \                |_   _| ___ \ /  ___|         | |                
# | |_/ / __ _ ___  ___    | | | |_/ / \ `--. _   _ ___| |_ ___ _ __ ___  
# | ___ \/ _` / __|/ _ \   | | |    /   `--. \ | | / __| __/ _ \ '_ ` _ \ 
# | |_/ / (_| \__ \  __/  _| |_| |\ \  /\__/ / |_| \__ \ ||  __/ | | | | |
# \____/ \__,_|___/\___|  \___/\_| \_| \____/ \__, |___/\__\___|_| |_| |_|
#                                             __/ |                      
#                                            |___/                       
# --------------------------------------------------------------------------------

topics = {}
judged_documents = {}
index_id = 1

# -----------------------------------------------------------------------------
# write_to_file() - Small auxiliary function to write data to a file
# -----------------------------------------------------------------------------
def write_to_file(dic, filename):
    with open('material/saved_data/{}.txt'.format(filename), 'wb') as write_f:
        pickle.dump(dic, write_f)
    return

# -----------------------------------------------------------------------------
# read_from_file() - Small auxiliary function to read data from a file
# -----------------------------------------------------------------------------
def read_from_file(filename):
    with open('material/saved_data/{}.txt'.format(filename), 'rb') as read_f:
        return pickle.load(read_f) 

#--------------------------------------------------
# get_xml_files_recursively - Auxiliary function to get_files_from_directory
#
# Input: path - The path to the parent directory or file from which to start our recursive function
#               
# Behaviour: Creates a list with the path to every file that's an hierarquical child of parent directory path,
# recursively going through each child in Post-Order traversing
#
# Output: A List with the paths to each file child
#--------------------------------------------------
def get_xml_files_recursively(path, judged_documents, **kwargs):
    files_list = []
    directory_list = os.listdir(path)
    for f in directory_list:
        n_path = '{}{}/'.format(path,f)
        if os.path.isdir(n_path):
            files_list.extend(get_xml_files_recursively(n_path, judged_documents, **kwargs))

        elif judged_documents != None:
                if int(f.split('news')[0]) in judged_documents:
                    files_list.append(re.sub('//','/','{}/{}'.format(path,f)))
        else:
            files_list.append(re.sub('//','/','{}/{}'.format(path,f)))
    return files_list

# -------------------------------------------------
# get_files_from_directory - Recursively gets all files from directory or file path, parsing the files from xml to objects
# and spliting them in D_Test and D_Train in the conditions specified by our project
#
# Input: path - The path to the parent directory or file from which to start our search
#               
# Behaviour: It starts by creating a list with the path to every file that's an hierarquical child of parent directory path,
# recursively going through each child in Post-Order traversing. Afterwards it parses each and every file from xml to a runtime
# object using the BeautifulSoup library. At last after having all files in object form it splits the dataset in D_Test and D_Train
# sets, according to their identifier (D_Test -> identifier > 1996-09-30   D_Train -> identifier <= 1996-09-30)
#
# Output: A List with the Lists of file objects present in D_Test and D_Train
# -------------------------------------------------
def get_files_from_directory(path, judged_documents, **kwargs):
    file_list = get_xml_files_recursively(path, judged_documents, **kwargs)

    parsed_files_test = []
    parsed_files_train = []

    if 'set' in kwargs and kwargs['set'] == 'test':
        for f in file_list:
            date_identifier = int(f.split('/')[2])

            if date_identifier <= 19960930:
                continue

            open_file = open(f, 'r')
            parsed_file = BeautifulSoup(open_file.read(), 'lxml')
            
            if parsed_file.copyright != None:
                parsed_file.copyright.decompose()

            if parsed_file.codes != None:
                parsed_file.codes.decompose()
                
            parsed_files_test += [parsed_file,]

    elif 'set' in kwargs and kwargs['set'] == 'train':
        for f in file_list:
            date_identifier = int(f.split('/')[2])

            if date_identifier > 19960930:
                break

            open_file = open(f, 'r')
            parsed_file = BeautifulSoup(open_file.read(), 'lxml')
            
            if parsed_file.copyright != None:
                parsed_file.copyright.decompose()

            if parsed_file.codes != None:
                parsed_file.codes.decompose()
                
            parsed_files_train += [parsed_file,]

    else:
        for f in file_list:
            date_identifier = int(f.split('/')[2])

            open_file = open(f, 'r')
            parsed_file = BeautifulSoup(open_file.read(), 'lxml')
            
            if parsed_file.copyright != None:
                parsed_file.copyright.decompose()

            if parsed_file.codes != None:
                parsed_file.codes.decompose()
                
            if date_identifier <= 19960930:
                parsed_files_train += [parsed_file,]
            else:
                parsed_files_test += [parsed_file,]

    return (parsed_files_test, parsed_files_train)

# -----------------------------------------------------------------------------------------------------------
# processing - Processes text in String form
#
# Input: text - The text in String form to be processed
#        **kwargs - Optional named arguments, with the following functionality (default values prefixed by *)
#               lowercasing [*True | False]: Flag to perform Lowercasing 
#               punctuation [*True | False]: Flag to remove punction
#               spellcheck [True | *False]: Flag to perform spell check using TextBlob
#               stopwords [*True | False]: Flag to remove Stop Words 
#               simplication [*lemmatization | stemming | None]: Flag to perform Lemmatization or Stemming
#               
# Behaviour: Procceses the text in the input argument text as refered to by the arguments in **kwargs,
# behaviour being completely dependent on them except for Tokenization which is always performed
#
# Output: A String with the processed text 
# ----------------------------------------------------------------------------------------------------------
def processing(text, **kwargs):

    p_text = text
    # Lowercasing the entire string
    if 'lowercasing' not in kwargs or kwargs['lowercasing']:
        p_text = p_text.lower()

    #Remove punctuation
    if 'punctuation' not in kwargs or kwargs['punctuation']:
        p_text = re.sub("[/-]"," ",p_text)
        p_text = re.sub("[.,;:\"\'!?`´()$£€]","",p_text)

    # Spell Check
    if "spellcheck" in kwargs and kwargs['spellcheck']:          
        p_text = str(TextBlob(p_text).correct())

    # Tokenization
    tokens = nltk.word_tokenize(p_text)
    string_tokens = ''

    # Spell Check correction
    if "spellcheck" in kwargs and kwargs['spellcheck']:
        n_tokens = []
        for word in tokens:           
            n_tokens += ' {}'.format(TextBlob(word).correct)

    # Lemmatization
    if 'simplification' not in kwargs or kwargs['simplification'] == 'lemmatization':
        lemma = WordNetLemmatizer()

        #Remove stopwords
        if 'stopwords' not in kwargs or kwargs['stopwords']:
            for word in tokens:
                if word not in stopwords.words('English'):   
                    string_tokens += ' {}'.format(lemma.lemmatize(word))
        else: 
            for word in tokens: 
                string_tokens += ' {}'.format(lemma.lemmatize(word))

    # Stemming
    elif kwargs['simplification'] == 'stemming':
        stemer = nltk.stem.snowball.EnglishStemmer()

        #Remove stopwords
        if 'stopwords' not in kwargs or kwargs['stopwords']:
            for word in tokens:
                if word not in stopwords.words('English'):   
                    string_tokens += ' {}'.format(stemer.stem(word))
        else: 
            for word in tokens: 
                string_tokens += ' {}'.format(stemer.stem(word))

    # Case for no simplification
    else:
        for word in tokens: 
            string_tokens += ' {}'.format(word)   

    # Removing the first whitespace in the output 
    return string_tokens[1:]

# -----------------------------------------------------------------------------
# process_collection - Small auxiliary function to externaly process a text 
# collection independently of program function
# -----------------------------------------------------------------------------

def process_collection(collection, tokenize, **kwargs):
    result = {}
    for doc in collection:
        item_id = int(doc.newsitem.get('itemid'))
        title = processing(re.sub('<[^<]+>', "", str(doc.title)), **kwargs)
        dateline = processing(re.sub('<[^<]+>|\w[0-9]+-[0-9]+-[0-9]+\w', "", str(doc.dateline)), **kwargs)
        text = processing(re.sub('<[^<]+>', "", str(doc.find_all('text')))[1:-1], **kwargs)
        
        if tokenize:
            result[item_id] = nltk.word_tokenize('{} {} {}'.format(title, dateline, text))
        else:
            result[item_id] = '{}\n{}\n{}'.format(title, dateline, text)

    return result

# -----------------------------------------------------------------------------------------------------
# tfidf_process - Processes our entire document collection with a tf-idf vectorizer 
# and transforms the entire collection into tf-idf spaced vectors 
#
# Input: doc_dic - The entire document collection in dictionary form
#        **kwargs - Optional parameters with the following functionality (default values prefixed by *)
#               norm [*l2 | l1]: Method to calculate the norm of each output row
#               min_df [*1 | float | int]: Ignore the terms which have a freq lower than min_df
#               max_df [*1.0 | float | int]: Ignore the terms which have a freq higher than man_df
#               max_features [*None | int]: 
#
# Behaviour: Creates a tf-idf vectorizer and fits the entire document collection into it. 
# Afterwards, transforms the entire document collection into vector form, allowing it to be 
# directly used to calculate similarities. It also converts structures into to an easy form to manipulate 
# at the previous higher level.
#
# Output: The tf-idf vectorizer created, a list of document keys (ids) and the entire doc
# collection in vector form.
# -----------------------------------------------------------------------------------------------------
def tfidf_process(doc_dic, **kwargs):
    doc_keys = list(doc_dic.keys())
    doc_list = []

    for doc in doc_keys:
        doc_list.append(doc_dic[doc])

    norm = 'l2' if 'norm' not in kwargs else kwargs['norm']
    min_df = 2 if 'min_df' not in kwargs else kwargs['min_df']
    max_df = 0.8 if 'max_df' not in kwargs else kwargs['max_df']
    max_features = None if 'max_features' not in kwargs else kwargs['max_features']
    stop_words = None if 'remove_stopwords' not in kwargs else kwargs['remove_stopwords']

    vec = TfidfVectorizer(norm=norm, min_df=min_df, max_df=max_df, max_features=max_features, stop_words= stop_words)
    vec.fit(doc_list)

    doc_list_vectors = vec.transform(doc_list)

    return [vec, doc_keys, doc_list_vectors]

# -----------------------------------------------------------------------
# get_topics - Auxiliary function that gathers info on all topics
#
# Input: directory - Directory path for project materials
# 
# Behaviour: Extracts topic info from '{directory}topics.txt' and updates
# the global dictionary which stores topic info
#
# Output: None
# -----------------------------------------------------------------------
def get_topics(directory):
    topics = {}
    
    topic_f = open('{}topics.txt'.format(directory), 'r')
    parsed_file = BeautifulSoup(topic_f.read(), 'lxml')

    topic_list = parsed_file.find_all('top')

    for topic in topic_list:
        split_topic = topic.getText().split('\n')
        split_topic = list(filter(lambda x: x!='', split_topic))

        number = split_topic[0].split(' ')[2][1:]
        title = processing(split_topic[1])
        topics[int(number)] = re.sub(' +',' ',title)
    return topics

# -------------------------------------------------------------------------------------------------
# get_R_set - Auxiliary function that extracts the R set
#
# Input: directory - Directory path for project materials
# 
# Behaviour: Extracts the triplet (Topic id, Document id, Feedback) for each entry in the 
# R set, present in '{directory}qrels_test.txt' (R-test) and '{directory}qrels_test.txt' (R-train)
#
# Output: [R-Test, R-Train], each being a list of triplet entries
# -------------------------------------------------------------------------------------------------
def get_R_set(directory, **kwargs):
    judged_documents = {}

    r_test_f = open('{}qrels_test.txt'.format(directory), 'r')
    r_train_f = open('{}qrels_train.txt'.format(directory), 'r')

    r_test_lines = r_test_f.readlines()
    r_train_lines = r_train_f.readlines()

    r_test_lines = [r_test_lines, r_train_lines]
    r_set = [{},{}]
    
    if 'index' in kwargs and kwargs['index'] == 'doc_id':
        for i in range(2):
            for line in r_test_lines[i]:
                split_entry = line.split(' ')
                topic_id = int(split_entry[0][1:])
                doc_id = int(split_entry[1])

                if doc_id not in judged_documents: 
                    judged_documents[doc_id] = True

                feedback = int(split_entry[2])

                if doc_id not in r_set[i]:
                    r_set[i][doc_id] = {}
                r_set[i][doc_id][topic_id] = feedback

    else:
        for i in range(2):
            for line in r_test_lines[i]:
                split_entry = line.split(' ')
                topic_id = int(split_entry[0][1:])
                doc_id = int(split_entry[1])

                if doc_id not in judged_documents: 
                    judged_documents[doc_id] = True

                feedback = int(split_entry[2])

                if topic_id not in r_set[i]:
                    r_set[i][topic_id] = {}
                r_set[i][topic_id][doc_id] = feedback

    return [r_set, judged_documents]

# -------------------------------------------------------------------------------------------------
# find_R_test_labels - Function that finds the test labels for a given R_Set
#
# Input: R_test - The R_Test set 
#
# Behaviour: Extrapolates the feedback from the R_Test set to a dic or array
#
# Output: The R_Test set labels in dic or np array form
# -------------------------------------------------------------------------------------------------
def find_R_test_labels(R_test, **kwargs):
    r_labels = None

    if 'list' not in kwargs:
        r_labels = {}
        for doc in R_test:
            r_labels[doc] = R_test[doc]

    elif 'list' in kwargs and kwargs['list']:
        r_labels = []
        for doc in R_test:
            r_labels.append(R_test[doc])

    return r_labels

# -----------------------------------------------------------------------------
# get_judged_docs - Small auxiliary function that returns the judged documents
# in the given rcv1 directory
# -----------------------------------------------------------------------------
def get_judged_docs(material_dir, rcv1_dir):
    judged_documents = get_R_set(material_dir)[1]
    return get_files_from_directory(rcv1_dir, judged_documents, judged=True)

# -------------------------------------------------------------------------------------------------
# find_ranked_query_labels - Function that finds the test labels for given query_docs and r_labels
#
# Input: query_docs - The ranked query docs
#        r_labels - the labels R_Test set produced 
#
# Behaviour: Compares de R_Test set feedback with the ranked docs
#
# Output: The labels for the ranked query docs in np array form
# -------------------------------------------------------------------------------------------------
def find_ranked_query_labels(query_docs, r_labels):
    q_docs = np.array(query_docs)
    q_docs = q_docs[:,0]

    query_labels = []
    result_labels = []

    for doc in query_docs:
        if doc[0] in r_labels:
            query_labels += [[doc[0], 1], ]
            result_labels += [[doc[0], r_labels[doc[0]]], ]
    
    for doc in r_labels:
        if doc not in q_docs:
            query_labels += [[doc, 0], ]
            result_labels += [[doc, r_labels[doc]], ]

    return [np.array(query_labels), np.array(result_labels)]  

# -------------------------------------------------------------------------------------------------
# find_boolean_query_labels - Function that finds the test labels for given query_docs and r_labels
#
# Input: query_docs - The query docs
#        r_labels - the labels R_Test set produced 
#
# Behaviour: Compares the R_Test set feedback with the ranked docs
#
# Output: The labels for the query docs in np array form
# -------------------------------------------------------------------------------------------------
def find_boolean_query_labels(query_docs, r_labels):
    query_labels = []
    result_labels = []

    for doc in r_labels:
        if doc in query_docs:
            query_labels += [[doc, 1], ]
            result_labels += [[doc, r_labels[doc]]]
        else:
            query_labels += [[doc, 0], ]
            result_labels += [[doc, r_labels[doc]]]

    return [np.array(query_labels), np.array(result_labels)]  

# -----------------------------------------------------------------------------
# normalize_dic() - Normalizes a dic 
# -----------------------------------------------------------------------------
def normalize_dic(dic, **kwargs):
    result_dic = {}

    values = dic.values()
    value_it = iter(values)

    if type(next(value_it)) == str:
        values_list = np.array([len(doc) for doc in values])
    else:
        values_list = np.array([score for score in values])
    
    if 'norm_method' not in kwargs or kwargs['norm_method'] == '1': 
        values_list = values_list / np.linalg.norm(values_list)

    elif kwargs['norm_method'] == '2':
        values_list = normalize(values_list[:,np.newaxis], axis=0).ravel()

    elif kwargs['norm_method'] == 'zscore':
        values_list = stats.zscore(values_list)

    keys = list(dic.keys())
    for i in range(len(keys)):
        result_dic[keys[i]] = values_list[i]

    return result_dic

# --------------------------------------------------------------------------------
# indexing - Creates an index after processing all text on data set D
#
# Input: D - The data set we will be building the index with
#        **kwargs - Optional named arguments for text preprocessing, with the following functionality (default values prefixed by *)
#               lowercasing [*True | False]: Flag to perform Lowercasing 
#               punctuation [*True | False]: Flag to remove punction
#               spellcheck [True | *False]: Flag to perform spell check using TextBlob
#               stopwords [*True | False]: Flag to remove Stop Words 
#               simplication [*lemmatization | stemming | None]: Flag to perform Lemmatization or Stemming
#               
# Behaviour: This function starts by creating the directory for our Index, after initializing our Schema fields. It then
# processes all documents on data set D and stores valuable information from them on the index (identifier, title, dateline and text).
# At last it commits the resulting processed documents to our index and calculates the total computational time the function used and the
# Disk space required to store the index.
#
# Output: A triplet tuple with the Inverted Index in object structure, the computational time for the function and 
# the disk space required to store the Inverted Index 
# --------------------------------------------------------------------------------
def indexing(D, **kwargs):
    global index_id

    start_time = time.time()
    ind_name = 'index{}'.format(str(index_id))
    ind_dir = '{}_dir'.format(ind_name)

    if os.path.exists(ind_dir):
        shutil.rmtree(ind_dir)
        os.mkdir(ind_dir)
    else:
        os.mkdir(ind_dir)

    schema = Schema(id= NUMERIC(stored=True), content= TEXT(stored=True))
    ind = index.create_in(ind_dir, schema=schema, indexname=ind_name)
    ind_writer = ind.writer()

    if not index.exists_in(ind_dir, indexname=ind_name):
        print("Error creating index")
        return

    processed_docs = process_collection(D, True, **kwargs)

    for doc in processed_docs:
        ind_writer.add_document(id=int(doc), content=processed_docs[doc])

    ind_writer.commit()
    
    time_required = round(time.time() - start_time, 6)
    
    space_required = os.path.getsize(ind_dir)

    return (ind, time_required, space_required)

# --------------------------------------------------------------------------------------------------------------------------------------
# extract_topic_query - Return the top-k informative terms from the topic q agains I using parameterizable scoring
#
# Input: q - The identifier number of the topic we want to search about
#        I - The Index object in which we will perform our search
#        k - The number of top-k terms to return 
#        **kwargs - Optional named arguments to parameterize scoring, with the following functionality (default values prefixed by *)
#               scoring [freq | tf-idf | dfree | pl2 |*bm25] - Chooses the scoring model we will use to score our terms
#               C [float | *1.0] - Free parameter for the pl2 model
#               B [float | *0.75] - Free parameter for the BM25 model
#               content_B [float | *1.0] - Free parameter specific to the content field for the BM25 model
#               k1 [float | *1.5] - Free parameter for the BM25 model
#
# Behaviour: Extracting the relevant model information from **kwargs, this function uses the index I present in its arguments 
# to perform a scored search on the top-k informative terms for topic q. It does so by creating a QueryParser object to parse
# the entire lenght of terms from q we've stored in our global topics structure and by using searcher.key_terms() to return
# the top terms according to our scoring weight vector. 
#
# Output: A List that contains the top k terms 
# -----------------------------------------------------------------------------------------------------------------------------------------
def extract_topic_query(q, I, k, **kwargs):
    global topics 
    topic = topics[q]

    topic_terms = []
    weight_vector = None

    # Chooses which score model to use from kwargs
    if 'scoring' not in kwargs:
        weight_vector = scoring.BM25F(B=0.75, content_B=1.0, K1=1.5)

    elif kwargs['scoring'] == 'freq':
        weight_vector = scoring.Frequency()

    elif kwargs['scoring'] == 'tf-idf':
        weight_vector = scoring.TF_IDF()

    elif kwargs['scoring'] == 'dfree':
        weight_vector = scoring.DFree()

    elif kwargs['scoring'] == 'pl2':
        C = 1.0 if 'C' not in kwargs else kwargs['C']

        weight_vector = scoring.PL2(c=C)

    elif kwargs['scoring'] == 'bm25':
        b = 0.75 if 'B' not in kwargs else kwargs['B']
        content_b = 1.0 if 'content_B' not in kwargs else kwargs['content_B']
        k1 = 1.5 if 'K1' not in kwargs else kwargs['K1']

        weight_vector = scoring.BM25F(B=b, content_B=content_b, K1=k1)    

    with I.searcher(weighting=weight_vector) as searcher:
        parser = QueryParser("content", I.schema, group=OrGroup).parse(topic)
        results = searcher.search(parser, limit=None)
        res_list = [int(r.values()[1]) for r in results]

        numbers_list = []
        for i in res_list:
            numbers_list += [searcher.document_number(id=i),]

        topic_terms = searcher.key_terms(numbers_list, "content", numterms=k, normalize=True)
      
    result = []
    for term in topic_terms:
        result += [term[0], ]

    return result

# -------------------------------------------------------------------------------------------------------------------
# boolean_query_aux - Auxiliary function to boolean_query that will check repeated ocurrences of documents
#
# Input: document_lists - A List of Lists in which each inner List has all documents in which the n-th term appeared
#        k - The number of terms we are using
#
# Behaviour: The function starts by calculating our error margin, in other words the number of missmatches a document
# can have before we stop considering it as relevant. This function composes a very simple algorithmn, where for each
# document we find in a sublist (non repeated, we use the list 'seen' to check that) we check if it's contained within 
# all other sublists, until it's not contained in miss_m + 1 lists. When that's the case, the document is no longer 
# relevant and we move on to the next one, iterating upon all elements of all sublists. The Time Complexity of this 
# function is O(N^2) while the Space Complexity is O(N)
#
# Output: A List of all relevants docs that don't exceed miss_m missmatches
# -------------------------------------------------------------------------------------------------------------------
def boolean_query_aux(document_lists, k):
    miss_m = round(0.2*k)
    seen = []
    result_docs = []

    for term_docs in document_lists:
        for doc in term_docs:
            if doc not in seen:
                chances = miss_m
                flag = True
                for doc_list in document_lists:
                    if doc not in doc_list:
                        if chances == 0:
                            flag = False
                            break
                        chances -= 1
                if flag:
                    result_docs += [doc,]
                seen += [doc, ]

    result_docs.sort()
    return result_docs

# ------------------------------------------------------------------------------------------
# boolean_query - Function that will query all documents in index I and find those who contain
# all top k-terms relevant to topic q allowing up to round(0.2*k) missmatches 
#
# Input: q - The identifier number of the topic we want to search about 
#        k - The number of top k-terms to check documents for
#        I - The Index object in which we will perform our search
#
# Behaviour: The function starts by running extract_topic_query to return top k-terms with which
# we will search for the relevant docs for topic q. Then we use the index I to perform a simple
# search on, parsing the result of our search per term to our auxiliary function. 
#
# Output: A List of all relevants docs that don't exceed miss_m missmatches
# ------------------------------------------------------------------------------------------
def boolean_query(q, k, I, **kwargs):
    terms = extract_topic_query(q, I, k, **kwargs)

    document_lists = []
    with I.searcher() as searcher:
        for term in terms:
            parser = QueryParser("content", I.schema, group=OrGroup).parse(term)
            results = searcher.search(parser, limit=None)
            term_list = [int(r.values()[1]) for r in results]
            document_lists += [term_list,]
            
    return boolean_query_aux(document_lists, k)


# ------------------------------------------------------------------------------------------
# cosine_scoring - Function that scores a document based on cosine similarity 
#
# Input: searcher - The searcher associated with the index I
#        all the other arguments are built-ins from FunctionWeighting() and old whoosh.scoring
#        documentation
#
# Behaviour: Uses the tf-idf result from searcher.idf() and applies cosine similarity formula
# to it
#
# Output: cosine similarity weight vector formula 
# ------------------------------------------------------------------------------------------
def cosine_scoring(searcher, fieldnum, text, docnum, weight, QTF=1):
    idf = searcher.idf(fieldnum, text)

    DTW = (1.0 + math.log(weight)) * idf
    QMF = 1.0
    QTW = ((0.5 + (0.5 * QTF/ QMF))) * idf
    return DTW * QTW

# -------------------------------------------------------------------------------------------------
# bpref - Function that runs the bpref evaluation metric
# -------------------------------------------------------------------------------------------------
def bpref(sol_labels):
    R = 0
    N = 0
    bpref = 0
    n_count = 0
    for label in sol_labels:
        if label == 0:
            N += 1
        else:
            R += 1

    for label in sol_labels:
        if label == 0:
            n_count += 1
        else:
            bpref += (1 - n_count/(min(R,N)))

    return (1 / R) * bpref

# -------------------------------------------------------------------------------------------------
# reciprocal_rank_fusion - Auxiliary function to calculate the RRF for the top-p documents
# Uses the formula RBF_score(f) = sum (1 / (50 + rank_f))
# -------------------------------------------------------------------------------------------------
def reciprocal_rank_fusion(p, ranking_lists):
    document_ranks = {}

    for rank_l in ranking_lists:
        for i in range(len(rank_l )):
            if rank_l[i][0] not in document_ranks:
                document_ranks[rank_l[i][0]] = 0
            document_ranks[rank_l[i][0]] += 1 / (50 + i+1)

    p_highest = None

    if p != None:
        p_highest = nlargest(p, document_ranks, key=document_ranks.get)
    else:
        p_highest = nlargest(len(document_ranks), document_ranks, key=document_ranks.get)
    
    results = []

    for p in p_highest:
        results += [[p, document_ranks[p]]]  

    return results

# ------------------------------------------------------------------------------------------------
# ranking - Function that will query all documents in index I and rank the top p ones
#
# Input: q - The identifier number of the topic we want to search about 
#        p - The number of top ranked documents we will return
#        I - The Index object in which we will perform our search
#        **kwargs - Optional named arguments to parameterize scoring, with the following functionality (default values prefixed by *)
#               ranking [cosine | RRF | tf-idf | *bm25] - Chooses the scoring model we will use to score our terms
#               B [float | *0.75] - Free parameter for the BM25 model
#               content_B [float | *1.0] - Free parameter specific to the content field for the BM25 model
#               k1 [float | *1.5] - Free parameter for the BM25 model
#
# Behaviour: The function uses the weight vector generated by its given scoring system to search and rank  
# the top-p documents in the index according to the full topic text.
#
# Output: A List of lists that contains pairs [document_id, score] in descending score ordering
# -------------------------------------------------------------------------------------------------
def ranking(q, p, I, **kwargs):
    global topics
    topic = topics[q]

    weight_vector = None
    if 'ranking' not in kwargs:
        weight_vector = scoring.BM25F(B=0.75, content_B=1.0, K1=1.5)

    elif kwargs['ranking'] == 'cosine':
        weight_vector = scoring.FunctionWeighting(cosine_scoring)

    elif kwargs['ranking'] == 'tf-idf':
        weight_vector = scoring.TF_IDF()

    elif kwargs['ranking'] == 'bm25':
        b = 0.75 if 'B' not in kwargs else kwargs['B']
        content_b = 1.0 if 'content_B' not in kwargs else kwargs['content_B']
        k1 = 1.5 if 'K1' not in kwargs else kwargs['K1']

        weight_vector = scoring.BM25F(B=b, content_B=content_b, K1=k1)  

    elif kwargs['ranking'] == 'RRF':
        bm25_ranking_1 = ranking(q, p, I, ranking="bm25")
        bm25_ranking_2 = ranking(q, p, I, ranking="bm25", b=0.5, content_b=1.25, k1=1.25)
        bm25_ranking_3 = ranking(q, p, I, ranking="bm25", b=0.5, content_b=1.5, k1=1.00)

        return reciprocal_rank_fusion(p, [bm25_ranking_1, bm25_ranking_2, bm25_ranking_3])

    with I.searcher(weighting=weight_vector) as searcher:
        parser = QueryParser("content", I.schema, group=OrGroup).parse(topic)
        results = searcher.search(parser, limit=p)
        
        term_list = []

        if p != None:
            for i in range(p):
                if i < len(results):
                    term_list += [(int(results[i].values()[1]), results.score(i)), ]
        else:
            for i in range(len(results)):
                term_list += [(int(results[i].values()[1]), results.score(i)), ]

    return term_list

# -------------------------------------------------------------------------------------------------
# evaluate_ranked_query - Auxiliary function to calculate statistical data
# -------------------------------------------------------------------------------------------------
def evaluate_ranked_query(topic, o_labels, sol_labels, **kwargs):
    results = {}

    results['accuracy'] = accuracy_score(sol_labels, o_labels)
    results['precision-micro'] = precision_score(sol_labels, o_labels, average='micro', zero_division=1)
    results['precision-macro'] = precision_score(sol_labels, o_labels, average='macro', zero_division=1)
    results['recall-micro'] =  recall_score(sol_labels, o_labels, average='micro')
    results['recall-macro'] =  recall_score(sol_labels, o_labels, average='macro')
    results['f-beta-micro'] = fbeta_score(sol_labels, o_labels, average='micro', beta=0.5)
    results['f-beta-macro'] = fbeta_score(sol_labels, o_labels, average='macro', beta=0.5)
    results['MAP'] = average_precision_score(sol_labels, o_labels)
    results['BPREF'] = bpref(sol_labels)

    if 'curves' in kwargs and kwargs['curves']:
        precision, recall, _ = precision_recall_curve(sol_labels, o_labels)
        PrecisionRecallDisplay(precision=precision, recall=recall).plot()
        plt.title('Precision Recall curve for Ranked topic {}'.format(topic))
        plt.show()

    return results

# -------------------------------------------------------------------------------------------------
# evaluate_boolean_query - Auxiliary function to calculate statistical data
# -------------------------------------------------------------------------------------------------
def evaluate_boolean_query(topic, o_labels, sol_labels, **kwargs):
    results = {}

    results['accuracy'] = accuracy_score(sol_labels, o_labels)
    results['precision-micro'] = precision_score(sol_labels, o_labels, average='micro', zero_division=1)
    results['precision-macro'] = precision_score(sol_labels, o_labels, average='macro', zero_division=1)
    results['recall-micro'] =  recall_score(sol_labels, o_labels, average='micro')
    results['recall-macro'] =  recall_score(sol_labels, o_labels, average='macro')
    results['f-beta-micro'] = fbeta_score(sol_labels, o_labels, average='micro', beta=0.5)
    results['f-beta-macro'] = fbeta_score(sol_labels, o_labels, average='macro', beta=0.5)
    results['MAP'] = average_precision_score(sol_labels, o_labels)

    
    if 'curves' in kwargs and kwargs['curves']:
        precision, recall, _ = precision_recall_curve(sol_labels, o_labels)
        PrecisionRecallDisplay(precision=precision, recall=recall).plot()
        plt.title('Precision Recall curve for Boolean topic {}'.format(topic))
        plt.show()


    return results

# -------------------------------------------------------------------------------------------------
# display_results_per_q - Auxiliary function to display calculated statistical data
# -------------------------------------------------------------------------------------------------
def display_results_per_q(q, results_ranked, results_boolean):
    print("Result for search on Topic {}".format(q))
    print("\nRanked Search:")
    for p in results_ranked:
        result_str= ''
        for m in results_ranked[p]:
            result_str += '{} = {}, '.format(m, round(results_ranked[p][m],4)) 
        print("For p={}: {}".format(p, result_str[:-2]))

    print("\nBoolean Search:")
    for k in results_boolean:
        result_str= ''
        for m in results_boolean[k]:
            result_str += '{} = {}, '.format(m, round(results_boolean[k][m],4)) 
        print("For k={}: {}".format(k, result_str[:-2]))

        print("Result for search on Topic {}".format(q))

    return
# -------------------------------------------------------------------------------------------------------
# evaluation - Function that fully evaluates our IR model, providing full statiscal analysis for several
# p and k values across multiple ranges and topics
#
# Input: Q_test - The set of topics we will evaluate the perform of our IR model on
#        R_test - The topic labels we are looking for
#        D_test - Our test set in collection form
#        **kwargs - The additional args in this function also refer to the additional args in indexing(),
#        ranking() and boolean_query(), for which documentation is provided above. Other than that, we have:
#               k_range [list of ints | *[1,2,4,6,8,10]] - List of k values our model will test
#               p_range [list of ints or None | *[100,200,300,400,500]] - List of p values our model will test
#               curves [True | *False] - Display the precision/recall curves
#
# Behaviour: The function provides full statistics for every topic in Q_test, using R_test and D_test
# to build an index. Then, for each p in p_range it will use ranking() to rank the top p documents
# and for each k in k_range it will use k to evaluate the relevant docs using boolean_query(). In the end,
# it uses retrival results to provide full statiscal analysis.
#
# Output: Full statistical analysis for the provided input args
# -----------------------------------------------------------------------------------------------------
def evaluation(Q_test, R_test, D_test, **kwargs):
    # Standard index execution
    I = indexing(D_test, **kwargs)[0]

    results_ranked = {}
    results_boolean = {}
    k_range = [1,2,4,6,8,10] if 'k_range' not in kwargs else kwargs['k_range']
    p_range = [100,200,300,400,500, None] if 'p_range' not in kwargs else kwargs['p_range']


    for q in Q_test:
        r_labels = find_R_test_labels(R_test[q])

        for p in p_range:
            score_docs = ranking(q, p, I, **kwargs)
            ranked_labels = find_ranked_query_labels(score_docs, r_labels)

            results_ranked[p] = evaluate_ranked_query(q, ranked_labels[0][:, 1],ranked_labels[1][:, 1], **kwargs)

        for k in k_range:
            boolean_docs = boolean_query(q, k, I, **kwargs)
            query_labels = find_boolean_query_labels(boolean_docs, r_labels)

            results_boolean[k] = evaluate_boolean_query(q, query_labels[0][:, 1], query_labels[1][:, 1], **kwargs)

    display_results_per_q(q, results_ranked, results_boolean)
    return

# --------------------------------------------------------------------------------------------
# overlapping_terms() - Function that finds the overlapping terms for a given k range
#
# Input: 
#
# Behaviour: Queries the top terms for all k's in a given k range and checks them for overlap
#
# Output: Data about the overlaping terms
# --------------------------------------------------------------------------------------------
def overlapping_terms():
    # Standard index execution
    I = index.open_dir("index_judged_docs_dir", indexname='index_judged_docs')

    k_range = [2,3,5,7,10,15]

    for k in k_range:
        top_terms = {}
        for q in range(101,201,1):
            results = extract_topic_query(q, I, k)
            for r in results:
                if r not in top_terms:
                    top_terms[r] = 0
                top_terms[r] += 1

        r_terms = 0
        for term in top_terms:
            if top_terms[term] > 1:
                r_terms += 1
        print("\nNumber of overlapping terms: {}".format(r_terms))
        print("Percent of overlapping terms: {}%".format(round(r_terms/len(top_terms)*100,3)))
        print(top_terms)
    return

# --------------------------------------------------------------------------------
#  _____  _              _               _               
# /  __ \| |            | |             (_)              
# | /  \/| | _   _  ___ | |_  ___  _ __  _  _ __    __ _ 
# | |    | || | | |/ __|| __|/ _ \| '__|| || '_ \  / _` |
# | \__/\| || |_| |\__ \| |_|  __/| |   | || | | || (_| |
# \____/ |_| \__,_||___/ \__|\___||_|   |_||_| |_| \__, |
#                                                   __/ |
#                                                  |___/            
# --------------------------------------------------------------------------------

tfidf_vec_info = [None, None, None]
Labels_pred = []

# --------------------------------------------------------------------------------
# get_clustering_score() - Gets a clustering methods score based on supervised, unsupervised
# or mixed metrics. Uses a linear combination of all results.
# --------------------------------------------------------------------------------
def get_clustering_score(x, labels_true, labels_pred, target):
    unsupervised_methods = { 'sil_score' : silhouette_score, 'calin_score' : calinski_harabasz_score, 'davies_score' : davies_bouldin_score}

    supervised_methods = {'rands_score' : adjusted_rand_score, 'v_score' : v_measure_score, 'complete_score' : completeness_score, 
                          'fowlkes_score' : fowlkes_mallows_score, 'homogenity_score': homogeneity_score, 'mutual_score' : mutual_info_score}

    if target == 'supervised':
        unsupervised_methods = {}
    elif target == 'unsupervised':
        supervised_methods = {}

    result = 0
    for method in unsupervised_methods:
        result += unsupervised_methods[method](x, labels_pred)

    for method in supervised_methods:
        score = supervised_methods[method](labels_true, labels_pred)
        result += score
        print('{} = {}'.format(method, score))

    return result / (len(unsupervised_methods) + len(supervised_methods))

# --------------------------------------------------------------------------------
# trainsKmeans - trains the KMeans algorithm
#
# Input: vec_D - the set of document or topics to be clustered
#       y - the set of ids and relevance info
#       clusters - the list with the number of clusters to be attempted
#       distance - parameter for the evaluation measures
# 
# Behaviour: creates the KMeans clusters for the number of clusters previously
# defined, and then applies a set of evaluation measures to select the best one
#
# Output: A list containing te best number of clusters to use, the list of labels
# and the rands score
# --------------------------------------------------------------------------------
def trainKmeans(vec_D, y, clusters, distance):

    array_D = vec_D.toarray()
    best_result = [None, None, 0, 0]
    for i in clusters:
        clustering_kmeans = KMeans(n_clusters=i).fit(vec_D)
        labels_pred = clustering_kmeans.labels_

        score_mean = get_clustering_score(array_D, y, labels_pred, 'unsupervised')

        if score_mean > best_result[2]:
            best_result = [clustering_kmeans, labels_pred, score_mean, i]  

    return best_result

# --------------------------------------------------------------------------------
# trainsAgglomerative - trains the Agglomerative Clustering algorithm
#
# Input: vec_D - the set of document or topics to be clustered
#       y - the set of ids and relevance info
#       clusters - the list with the number of clusters to be attempted
#       distance - parameter for the evaluation measures
# 
# Behaviour: creates the Agglomerative clusters for the number of clusters previously
# defined, and then applies a set of evaluation measures to select the best one
#
# Output: A list containing te best number of clusters to use, the list of labels
# and the rands score
# --------------------------------------------------------------------------------
def trainAgglomerative(vec_D, y, clusters, distance):

    vec_D = vec_D.toarray()
    best_result = [None, None, 0, 0]
    for i in clusters:
        clustering_agg = AgglomerativeClustering(n_clusters=i).fit(vec_D)
        labels_pred = clustering_agg.labels_

        score_mean = get_clustering_score(vec_D, y, labels_pred, 'unsupervised')

        if score_mean > best_result[2]:
            best_result = [clustering_agg, labels_pred, score_mean, i]

    return best_result

# ----------------------------------------------------------------------------------------------------
# get_topic_subset() - Retrieves topics from the global variable topics that are contained within
# Q 
# ----------------------------------------------------------------------------------------------------
def get_topic_subset(q_test):
    result = {}
    for topic in topics:
        if topic in q_test:
            result[topic] = topics[topic]
    return result

# ----------------------------------------------------------------------------------------------------
# get_category_ids() - Helper function to get category id's from collection D to serve as ground
# truth
# ----------------------------------------------------------------------------------------------------
def get_category_ids(doc_keys):
    clustered_docs = {}
    for doc in doc_keys:
        clustered_docs[doc] = True

    D = get_files_from_directory('../rcv1/', clustered_docs)
    D[0].extend(D[1])
    D = D[0]

    codes_y = []
    for doc in D:
        codes_y.append([])
        if len(doc.find_all('codes')) > 1:
            topic_section = doc.find_all('codes')[1]
            topic_ids = topic_section.find_all('code')

            for iden in topic_ids:
                codes_y[-1].append(iden['code'])
    codes_y = np.array(codes_y, dtype=object)

    return codes_y

# --------------------------------------------------------------------------------
#  _____                                   _                 _ 
# /  ___|                                 (_)               | |
# \ `--.  _   _  _ __    ___  _ __ __   __ _  ___   ___   __| |
#  `--. \| | | || '_ \  / _ \| '__|\ \ / /| |/ __| / _ \ / _` |
# /\__/ /| |_| || |_) ||  __/| |    \ V / | |\__ \|  __/| (_| |
# \____/  \__,_|| .__/  \___||_|     \_/  |_||___/ \___| \__,_|
#               | |                                            
#               |_|                                             
# --------------------------------------------------------------------------------

d_train = {}
d_test = {}
r_train = {}
r_test = {}
topic_vectorizers = {}

best_params = []

# -----------------------------------------------------------------------------------------------------
# create_vectorizer - Processes our entire document collection with a vectorizer 
# and transforms the entire collection into spaced vectors 
#
# Input: doc_dic - The entire document collection in dictionary form
#        feature_space - String defining which vectorizer is used (tf, idf or tf-idf)
#        **kwargs - Optional parameters with the following functionality (default values prefixed by *)
#               norm [*l2 | l1]: Method to calculate the norm of each output row
#               min_df [*1 | float | int]: Ignore the terms which have a freq lower than min_df
#               max_df [*1.0 | float | int]: Ignore the terms which have a freq higher than man_df
#               max_features [*None | int]: 
#
# Behaviour: Creates a vectorizer (tf, idf or tf-idf) and fits the entire document collection into it. 
# Afterwards, transforms the entire document collection into vector form, allowing it to be 
# directly used to calculate similarities. It also converts structures into to an easy form to manipulate 
# at the previous higher level.
#
# Output: The created Vectorizer and the entire doc collection in vector form.
# -----------------------------------------------------------------------------------------------------

def create_vectorizer(doc_dic, feature_space, **kwargs):
    doc_keys = list(doc_dic.keys())
    doc_list = []

    for doc in doc_keys:
        doc_list.append(doc_dic[doc])

    norm = 'l2' if 'norm' not in kwargs else kwargs['norm']
    min_df = 2 if 'min_df' not in kwargs else kwargs['min_df']
    max_df = 0.8 if 'max_df' not in kwargs else kwargs['max_df']
    max_features = None if 'max_features' not in kwargs else kwargs['max_features']
    stop_words = 'english' if 'remove_stopwords' not in kwargs else kwargs['remove_stopwords']
    vec = None
    doc_list_vectors = None

    if feature_space == 'tf':
        vec = CountVectorizer(min_df=min_df, max_df=max_df, max_features=max_features, stop_words= stop_words)
        vec.fit(doc_list)
        vec.transform(doc_list)
        
    elif feature_space == 'idf':
        vec = []
        vec.append(CountVectorizer(min_df=min_df, max_df=max_df, max_features=max_features, stop_words= stop_words))
        aux_vectors = vec[0].fit_transform(doc_list)

        vec.append(TfidfTransformer(smooth_idf=True, use_idf=True)) 
        doc_list_vectors = vec[1].fit_transform(aux_vectors)

    elif feature_space == 'tf-idf':
        vec = TfidfVectorizer(norm=norm, min_df=min_df, max_df=max_df, max_features=max_features, stop_words= stop_words)

    if type(vec) != list:
        vec.fit(doc_list)
        doc_list_vectors = vec.transform(doc_list)

    return [vec, doc_list_vectors]

# -------------------------------------------------------------------------------------------------
# display_results - Auxiliary function to display calculated statistical data
# -------------------------------------------------------------------------------------------------
def display_results(q, results):
    print("Result for search on Topic {}".format(q))
    result_str= ''
    for m in results:
        if m != 'ranking':
            result_str += '{} = {}, '.format(m, round(results[m],4)) 
    print("{}\n".format(result_str[:-2]))
  
    if 'ranking' in results:
        print("Top ranked documents on Topic {}".format(q))
        print("{}".format(results['ranking']))

    return
# --------------------------------------------------------------------------------
#  _____                     _      ______               _     _               
# |  __ \                   | |     | ___ \             | |   (_)              
# | |  \/ _ __  __ _  _ __  | |__   | |_/ / __ _  _ __  | | __ _  _ __    __ _ 
# | | __ | '__|/ _` || '_ \ | '_ \  |    / / _` || '_ \ | |/ /| || '_ \  / _` |
# | |_\ \| |  | (_| || |_) || | | | | |\ \| (_| || | | ||   < | || | | || (_| |
# \____/|_|   \__,_|| .__/ |_| |_| \_| \_|\__,_||_| |_||_|\_\|_||_| |_| \__, |
#                   | |                                                  __/ |
#                   |_|                                                 |___/                       
# --------------------------------------------------------------------------------

# -------------------------------------------------------------------------------------
# manhattan_distance_dic - Computes the manhattan distance between a document
# and a given list of documents
#
# Input: doc_query - Document that serves as basis to compare all other documents to
#        vectorizer - The structure that contains our tf-idf vectorizer
#        doc_keys - A list of all document keys
#        doc_vectors - All documents in vector form contained in the vectorizer space
#        theta - The similarity threshold 
#
# Behaviour: It starts by transforming the query document into its vector notion in the
# vectorizer space. Afterwards, it calculates pairwise similarity based on the inverse 
# manhattan distance between all document vectors. In the end returns a dictionary with 
# all documents that have their similarity values (1/distance) greater than or equal to theta.
#
# Output: Dictionary with all documents that pass the similarity treshold
# -------------------------------------------------------------------------------------
def manhattan_distance_dic(doc_query, vectorizer, doc_keys, doc_vectors, theta, **kwargs):
    result = {}

    doc_vector = vectorizer.transform(doc_query)
    distance_list = manhattan_distances(doc_vector, doc_vectors)[0]

    for i in range(len(distance_list)):
        if distance_list[i] != 0 and 1/distance_list[i] >= theta:
            result[int(doc_keys[i])] = 1/distance_list[i]

    return result

# -----------------------------------------------------------------------------
# eucledian_distance_dic -  Computes the eucledian distance between a document
# and a given list of documents
#
# Input: doc_query - Document that serves as basis to compare all other documents to
#        vectorizer - The structure that contains our tf-idf vectorizer
#        doc_keys - A list of all document keys
#        doc_vectors - All documents in vector form contained in the vectorizer space
#        theta - The similarity threshold 
#
# Behaviour: It starts by transforming the query document into its vector notion in the
# vectorizer space. Afterwards, it calculates pairwise similarity based on the inverse 
# eucledian distance between all document vectors.  In the end returns a dictionary with 
# all documents that have their similarity values (1/distance) greater than or equal to theta.
#
# Output: Dictionary with all documents that pass the similarity treshold
# -----------------------------------------------------------------------------
def eucledian_distance_dic(doc_query, vectorizer, doc_keys, doc_vectors, theta, **kwargs):
    result = {}

    doc_vector = vectorizer.transform(doc_query)
    distance_list = euclidean_distances(doc_vector, doc_vectors)[0]

    for i in range(len(distance_list)):
        if distance_list[i] != 0 and 1/distance_list[i] >= theta:
            result[int(doc_keys[i])] = 1/distance_list[i]

    return result

# -----------------------------------------------------------------------------
# cosine_similarity_dic - Computes the cosine similarity between a document
# and a given list of documents
#
# Input: doc_query - Document that serves as basis to compare all other documents to
#        vectorizer - The structure that contains our tf-idf vectorizer
#        doc_keys - A list of all document keys
#        doc_vectors - All documents in vector form contained in the vectorizer space
#        theta - The similarity threshold 
#
# Behaviour: It starts by transforming the query document into its vector notion in the
# vectorizer space. Afterwards, it calculates pairwise similarity based on the cosine 
# similarity measure between all document vectors. In the end returns a dictionary with 
# all documents that have their similarity values greater than or equal to theta.
#
# Output: Dictionary with all documents that pass the similarity treshold
# -----------------------------------------------------------------------------
def cosine_similarity_dic(doc_query, vectorizer, doc_keys, doc_vectors, theta, **kwargs):
    result = {}

    doc_vector = vectorizer.transform(doc_query)
    distance_list = cosine_similarity(doc_vector, doc_vectors)[0]

    for i in range(len(distance_list)):
        if distance_list[i] >= theta:
            result[int(doc_keys[i])] = distance_list[i]

    return result

# ------------------------------------------------------------------------------
# sim_method_helper - Short helper method to encapsulate choosing the correct
# similarity function
#
# Input: sim - The string which represents which similarity function to use
#
# Behaviour: Matches the string with a dictionary of functions we have available
#
# Output: The function to use
# ------------------------------------------------------------------------------
def sim_method_helper(sim):
    sim_methods = {'cosine': cosine_similarity_dic, 'euclidean': eucledian_distance_dic, 'manhattan': manhattan_distance_dic}
    sim_method = None
    
    if sim == None:
        sim_method = sim_methods['cosine']

    elif sim == 'cosine' or sim == 'euclidean' or sim == 'manhattan':
        sim_method = sim_methods[sim]
    
    else:
        print("Error: similarity measure not recognized.")

    return sim_method

# ------------------------------------------------------------------------------------------------
# ranking_for_page_rank - Function that uses our ranking function to format data for page_rank
# non-uniform priors
#
# Input: query - The query we are searching our index on 
#        p - The number of top ranked documents we will return
#        D - A document collection 
#        **kwargs - Optional named arguments to parameterize scoring, with the following functionality (default values prefixed by *)
#               method [*None | len ] - Chooses a method to calculate priors
#               
# Behaviour: Uses or original IR system or the documents lenght to calculate non-uniform priors
#
# Output: A dictionary with the top p entries in the form doc_id : score
# -------------------------------------------------------------------------------------------------
def ranking_page_rank(query, p, D, **kwargs):
    from base_IRsystem import indexing
    result_dic = {}

    if 'prior_method' not in kwargs:
        I = indexing(D, **kwargs)[0]
        

        with I.searcher(weighting=scoring.BM25F(B=0.75, content_B=1.0, K1=1.5)) as searcher:
            parser = QueryParser("content", I.schema, group=OrGroup).parse(query)
            results = searcher.search(parser, limit=p)

            if p != None:
                for i in range(p):
                    if i < len(results):
                        result_dic[int(results[i].values()[1])] = results.score(i)
    
    elif kwargs['prior_method'] == 'len':
        processed_collection = process_collection(D, False)
        result_dic = processed_collection 

    return normalize_dic(result_dic)

# ---------------------------------------------------------------------------------------------------------
# page_rank - Function that directly uses the Page Rank algorithm with a 
# variation for undirected graphs and uses it to calculate a score for each
# candidate based on the provided link_graph. 
#
# Input: link_graph - The undirected graph that contains all document links
#        and their correspondent weight.
#        q - A topic query in the form of topic identifier (int)
#        D - The document collection we built our graph with
#        **kwargs - Optional parameters with the following functionality (default values prefixed by *)
#               iter [*50 | int]: Number of iterations to run the algorithm in 
#               p [*0.15 | float]: Starting p value which represents the residual probability for each node
#               prior [*uniform | non-uniform]: Method to calculate priors in our algorithm 
# 
# Behaviour: This function starts by setting the default values for the Page Rank algorithm, and after 
# selecting which prior to use, it applies the algorithm max_iters number of times. It also builds some
# auxiliary structures like link_count to ensure we don't repeatly calculate const values. 
#
# Output: The resulting PageRank graph in dictionary form. 
# ---------------------------------------------------------------------------------------------------------
def page_rank(link_graph, q, D, **kwargs):
    result_graph = {}

    max_iters = 50 if 'iter' not in kwargs else kwargs['iter']
    p = 0.15 if 'p' not in kwargs else kwargs['p']
    N = len(link_graph)
    follow_p = 1 - p
    prior = None

    if 'prior' not in kwargs or kwargs['prior'] == 'uniform':
        prior = p / N

        # Setting uniform priors
        for doc in link_graph:
            result_graph[doc] = prior

        # Dictionary to save max_iters * (N-1) len() operations
        link_count = {}
        for doc in link_graph:
            link_count[doc] = len(link_graph[doc])

        for _ in range(max_iters):
            iter_graph = {}

            for doc in result_graph:
                cumulative_post = 0

                for link in link_graph[doc]:
                    cumulative_post += (result_graph[link] / link_count[doc]) 
                iter_graph[doc] = prior + follow_p * cumulative_post 

            result_graph = iter_graph

    elif 'prior' in kwargs and kwargs['prior'] == 'non-uniform':

        ranked_dic = ranking_page_rank(topics[q], len(link_graph), D, **kwargs)
        prior_dic = {}

        # Initialize prior using original IR system
        for doc in link_graph:
            if doc in ranked_dic:
                prior_dic[doc] = ranked_dic[doc]
            else:
                prior_dic[doc] = 0

        result_graph = deepcopy(prior_dic)

        # Dictionary to save max_iters * (N-1) cum_sum operations
        link_weighted_count = {}
        for doc in link_graph:
            link_weighted_count[doc] = 0
            for link in link_graph[doc]:
                link_weighted_count[doc] += link_graph[doc][link]

        for _ in range(max_iters):
            iter_graph = {}

            for doc in result_graph:
                cumulative_prior = 0
                cumulative_post = 0

                for link in link_graph[doc]:
                    cumulative_prior += prior_dic[link]
                    cumulative_post += ((result_graph[link] * link_graph[link][doc]) / link_weighted_count[doc]) 

                iter_graph[doc] = p * cumulative_prior + follow_p * cumulative_post 

            result_graph = iter_graph

    return result_graph

# -------------------------------------------------------------------------------------------------
# evaluate_page_rank - Auxiliary function to calculate statistical data
# -------------------------------------------------------------------------------------------------
def evaluate_page_rank(topic, o_labels, sol_labels, **kwargs):
    results = {}

    results['accuracy'] = accuracy_score(sol_labels, o_labels)
    results['precision-micro'] = precision_score(sol_labels, o_labels, average='micro', zero_division=1)
    results['precision-macro'] = precision_score(sol_labels, o_labels, average='macro', zero_division=1)
    results['recall-micro'] =  recall_score(sol_labels, o_labels, average='micro')
    results['recall-macro'] =  recall_score(sol_labels, o_labels, average='macro')
    results['f-beta-micro'] = fbeta_score(sol_labels, o_labels, average='micro', beta=0.5)
    results['f-beta-macro'] = fbeta_score(sol_labels, o_labels, average='macro', beta=0.5)
    results['MAP'] = average_precision_score(sol_labels, o_labels)

    if 'curves' in kwargs and kwargs['curves']:
        precision, recall, _ = precision_recall_curve(sol_labels, o_labels)
        PrecisionRecallDisplay(precision=precision, recall=recall).plot()
        plt.title('Precision Recall curve for Ranked topic {}'.format(topic))
        plt.show()

    return results

# -------------------------------------------------------------------------------------------------
# display_results_page_rank - Auxiliary function to display calculated statistical data
# -------------------------------------------------------------------------------------------------
def display_results_page_rank(q, results_page_rank):
    print("\nPage Rank Search:")
    for theta in results_page_rank:
        result_str= ''
        for m in results_page_rank[theta]:
            result_str += '{} = {}, '.format(m, round(results_page_rank[theta][m],4)) 
        print("For theta={}: {}".format(theta, result_str[:-2]))

    return

# -------------------------------------------------------------------------------------------------------
# evaluation - Function that fully evaluates our IR model, providing full statiscal analysis for several
# theta values across multiple topics
#
# Input: Q_test - The set of topics we will evaluate the perform of our IR model on
#        R_test - The topic labels we are looking for
#        D_test - Our test set in collection form
#        **kwargs - Optional parameters with the following functionality (default values prefixed by *)
#               theta_range [list of floats or None | *[100,200,300,400,500]] - List of theta values our model will test
#               sim_method [*cosine | eucledian | manhattan] - Sim method our page rank graph will use
#
# Behaviour: The function provides full statistics for every topic in Q_test, using R_test and D_test.
# For each theta in theta_range it will use undirected_page_rank() to rank the top p documents.
#
# Output: Full statistical analysis for the provided input args
# -----------------------------------------------------------------------------------------------------
def evaluation_gr(Q_test, R_test, D_test, **kwargs):

    results_page_rank = {}
    theta_range = [0.20, 0.25, 0.30, 0.35, 0.40] if 'theta_range' not in kwargs else kwargs['theta_range']
    sim_method = 'cosine' if 'sim_method' not in kwargs else kwargs['sim_method']

    for q in Q_test:
        r_labels = find_R_test_labels(R_test[q])

        for theta in theta_range:
            page_rank_docs = undirected_page_rank(q, D_test, 100, sim_method, theta, **kwargs)
            ranked_labels = find_ranked_query_labels(page_rank_docs, r_labels)

            results_page_rank[theta] = evaluate_page_rank(q, ranked_labels[0][:, 1],ranked_labels[1][:, 1], **kwargs)
            
        display_results_page_rank(q, results_page_rank)
        
    return

The previous section composes the enterity of our code that's not directly tied to the main functions for each topic, including every auxiliary function.

# 3.1 - Clustering approach: organizing collections

 **clustering(D, kwargs)** Applies Clustering, a unsupervised learning technique to classify 
 sets of documents instead of individual documents

 **Input:** (D, kwargs)
 
       D - set of documents or topics to be clustered
       
       kwargs - Optional parameters with the following functionality (default 
       values prefixed by *)
           clusters [*range(2,20) | list of ints > 0]: List with the number of clusters to attempt in 
           the clustering algorithms
           distance [*euclidian | manhattan]: Distance measurement used 
           top_cluster_words [*5 | int]: Number of top words that represent each cluster 

 **Behaviour:** Starts by obtaining the processed collection of documents. Then vectorizes
 them throught the function tfidf_process and obtains the vectorizer itself, the document
 keys and the document vector. Afterwards, we obtain the r_set entries relevant to the doc keys,
 and deal with the kwargs information. These arguments are sent to the clustering training functions
 which return the information regarding the best clustering they found. We compare KM and AC through
 their best rand score, and then process the information to output.

 **Output:** A list of cluster results. These results consist in a pair per cluster, with the cluster 
 centroid and the set of document/topic ids which comprise it


In [33]:
def clustering(D, **kwargs):
    global tfidf_vec_info
    global labels_pred

    mode = 'docs' if 'mode' not in kwargs else kwargs['mode']

    doc_keys = None
    doc_vectors = None
    y = []

    if mode == 'docs':
        tfidf_vec_info = tfidf_process(D, remove_stopwords='english', **kwargs)
        doc_keys = tfidf_vec_info[1]
        doc_vectors = tfidf_vec_info[2]

        r_set = get_R_set('material/',index='doc_id')[0][1]
    
        for i in range(len(doc_keys)):
            y.append([])
            if doc_keys[i] in r_set:
                for r in r_set[doc_keys[i]]:
                    y[i].append('{}_{}'.format(r, r_set[doc_keys[i]][r]))
        y = np.array(y, dtype=object)

    elif mode == 'topics':
        tfidf_vec_info = tfidf_process(D, min_df = 1, max_df=1.0, **kwargs)
        doc_keys = tfidf_vec_info[1]
        doc_vectors = tfidf_vec_info[2]

        r_set = get_R_set('material/')[0][1]

        for i in range(len(doc_keys)):
            y.append([])
            if doc_keys[i] in r_set:
                for r in r_set[doc_keys[i]]:
                    y[i].append('{}_{}'.format(r, r_set[doc_keys[i]][r]))
        y = np.array(y, dtype=object)

    clustering_methods = [trainKmeans, trainAgglomerative] if 'methods' not in kwargs else kwargs['methods']

    clusters = list(range(2,100)) if 'clusters' not in kwargs else kwargs['clusters']
    distances = ['euclidean'] if 'distance' not in kwargs else kwargs['distance']
    top_cluster_words = 5 if 'top_cluster_words' not in kwargs else kwargs['top_cluster_words']
    
    # [Object, Labels, Score, n_clusters]
    best_clusters = [None, None, 0, 0]

    for method in clustering_methods:
        for dist in distances:
            clustering = method(doc_vectors, y, clusters, dist)

            if clustering[2] > best_clusters[2]:
                best_clusters = clustering

    result = []
    doc_labels = best_clusters[1]
    labels_pred = best_clusters[1]

    for i in range(best_clusters[3]):
        result.append([None,[]])

    centroids = None

    if type(best_clusters[0]) == KMeans:
        centroids = np.array(best_clusters[0].cluster_centers_)

        for i in range(len(doc_keys)):
            centroid = doc_labels[i]
            result[centroid][1].append(doc_keys[i])

    else:
        docs_per_centroid = {}
        for i in range(len(doc_keys)):
            centroid = doc_labels[i]
            result[centroid][1].append(doc_keys[i])

            if centroid not in docs_per_centroid:
                docs_per_centroid[centroid] = [doc_vectors[i].toarray()[0]]
            else:
                docs_per_centroid[centroid].append(doc_vectors[i].toarray()[0])

        centroids = []
        for centroid in docs_per_centroid:
            value = np.mean(docs_per_centroid[centroid], axis=0)
            centroids.append(value)

    for i in range(len(centroids)):
        aux_centroid = centroids[i][centroids[i] != 0.]
        sorted_args = np.argsort(aux_centroid)

        inverse_transformed = tfidf_vec_info[0].inverse_transform(centroids[i])[0]
        n_args = len(inverse_transformed)
        entry_range = top_cluster_words if n_args > top_cluster_words else n_args

        centroid_result = []
        for j in range(entry_range):
            centroid_result.append(inverse_transformed[sorted_args[j]])

        entry = centroid_result
        result[i][0] = entry

    return result


# Some simple example executions
# Change this to given rcv1 directory. Small sample size
D_set = get_files_from_directory('../rcv1_test/19960821/', None)[1]
D_set = process_collection(D_set, False)

print('\nDocument set results')
print(clustering(D_set, method = [trainKmeans], clusters=list(range(2,5)), top_cluster_words=5))
print(clustering(D_set, method = [trainAgglomerative], clusters=list(range(5,7)), top_cluster_words=5))

material_dic = 'material/'
topics = get_topics(material_dic)
Q_set = get_topic_subset(list(range(101,121)))

print('\nTopic set results')
print(clustering(Q_set, method = [trainKmeans], clusters=list(range(2,5)), top_cluster_words=5, mode='topics'))
print(clustering(Q_set, method = [trainAgglomerative], clusters=list(range(5,7)), top_cluster_words=5, mode='topics'))


Document set results
[[['helped', 'growing', 'try', 'damage', 'hawaii'], [4949, 4951, 4952, 4955, 4956, 4957, 4958, 4961, 4962, 4963]], [['high', 'day', '10', 'say', 'market'], [4948, 4950, 4953, 4954, 4959, 4960]]]
[[['stock', 'world', 'added', 'plan', 'called'], [4952, 4955, 4959, 4962]], [['maker', 'corp', 'administration', 'friday', 'thursday'], [4948, 4960]], [['high', 'day', '10', 'say', 'market'], [4949, 4951, 4956, 4961, 4963]], [['market', 'high', 'maker', 'rose', 'world'], [4957, 4958]], [['year', 'usa', '10', 'high', 'time'], [4950, 4953, 4954]]]

Topic set results
[[['weight', 'loss', 'drug', 'harmful', 'east'], [101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119]], [['government', 'supported', 'voucher', 'death', 'mining'], [106, 112, 120]]]
[[['boat', 'convict', 'ferry', 'offender', 'repeat'], [108, 111, 113, 114, 115, 116, 117, 118]], [['case', 'custody', 'kidnapped', 'rescue', 'child'], [101, 102, 103, 105, 119]], [['government', 'suppo

**interpret(cluster, D, kwargs)** Outputs the representation of a cluster cointaining a median (centroid) and a medoid 

 **Input:** (cluster, D, kwargs)
 
            cluster - A document/topic cluster
            D - Set of documents or topics in cluster
            kwargs - Optional parameters with the following functionality (default 
            values prefixed by *)

 **Behaviour:** Creates a TF-IDF Vectorizer to process pairwise distances between each document in
 a cluster to calculate its medoid, and uses previous information to represent the centroid.

 **Output:** A list with centroid representation in top cluster words, and a medoid represented by
 a document or topic id 

In [34]:
def interpret(cluster, D, **kwargs):
    documents = {}
    for doc_id in cluster[1]:
        documents[doc_id] = D[doc_id]

    doc_vectors = tfidf_process(documents, min_df = 1, max_df=1.0, **kwargs)[2]
    distance_matrix = pairwise_distances(doc_vectors)

    centroid = cluster[0]
    medoid_index = np.argmin(distance_matrix.sum(axis=0))

    return [centroid, cluster[1][medoid_index]]

# Some simple example executions
Q_set = get_topic_subset(list(range(121,141)))

clustering_set_1 = clustering(Q_set, method = [trainKmeans], clusters=list(range(2,5)), top_cluster_words=5, mode='topics')
clustering_set_2 = clustering(Q_set, method = [trainAgglomerative], clusters=list(range(5,7)), top_cluster_words=5, mode='topics')
sets = [clustering_set_1, clustering_set_2]

print('\nIntrepet clustering results')
for clust_set in sets:
    print('\nExample set')
    for cluster in clust_set:
        print(interpret(cluster, Q_set))


Intrepet clustering results

Example set
[['mad', 'jakob', 'cow', 'creutzfeldt', 'britain'], 132]
[['china', 'missile', 'pakistan', 'plant', 'nuclear'], 121]
[['labor', 'law', 'television', 'child'], 128]
[['anti', 'drug', 'rejection', 'organ', 'pig'], 133]

Example set
[['drug', 'anti', 'rejection', 'problem', 'planning'], 139]
[['labor', 'law', 'television', 'child'], 128]
[['china', 'missile', 'pakistan', 'plant', 'nuclear'], 121]
[['fire', 'friendly', 'sea', 'turtle', 'death'], 132]
[['britain', 'great', 'statistic', 'abuse', 'substance'], 134]
[['cow', 'creutzfeldt', 'jakob', 'mad', 'parkinson'], 122]


**evaluate(D, kwargs)** Serves as the main class for the clustering process and outputs its results 

 **Input:** (D, kwargs)
 
         D - Document or topic collection to use
         kwargs - Optional parameters with the following functionality (default 
         values prefixed by *)
            mode [*docs | topics]: Chooses if we are running a topic or document collection 

**Behaviour:** It encapsulates the entirity of the clustering process and prints its output results
ordered by the result of each individual cluster.

**Output:** None

In [35]:
def evaluate(D, **kwargs):
    global tfidf_vec_info
    global labels_pred

    mode = 'docs' if 'mode' not in kwargs else kwargs['mode']
    doc_dic = D

    if mode == 'docs':
        doc_dic = process_collection(D, False, **kwargs)
    elif mode == 'topics':
        doc_dic = get_topic_subset(D)

    clusters = clustering(doc_dic, **kwargs)
    cluster_info = []

    for cluster in clusters:
        cluster_info.append(interpret(cluster, doc_dic, **kwargs))

    n_clusters = len(clusters)
    name = 'document' if mode == 'docs' else 'topic'

    print("The clustering solution has k = {} clusters".format(n_clusters))
    for i in range(n_clusters):
        print("\nCluster {}:".format(i+1))
        print("Centroid has top words {}".format(cluster_info[i][0]))
        print("Medoid is {} with id {}".format(name,cluster_info[i][1]))
        print("Cluster is composed by {} {}s".format(len(clusters[i][1]), name))
        print("{}s in cluster -> {}".format(name, clusters[i][1]))

    if mode == 'docs' and 'external' in kwargs and kwargs['external']:
        print('\nExternal evaluation for clustering solution:')
        doc_keys = tfidf_vec_info[1]
        category_ids = get_category_ids(doc_keys)
        final_score = get_clustering_score(None, category_ids, labels_pred, 'supervised')
        print('\nFinal Mean supervised score = {}'.format(final_score))
    
    return

# Some simple example executions
D_set = get_files_from_directory('../rcv1_test/19960821/', None)[1]
Q_set = list(range(141,161))

print('Example Execution with documents\n')
evaluate(D_set, clusters=list(range(3,5)))

print('\nExample Execution with topics\n')
evaluate(Q_set, clusters=list(range(4,6)), mode='topics')

Example Execution with documents

The clustering solution has k = 3 clusters

Cluster 1:
Centroid has top words ['market', 'high', 'maker', 'rose', 'beat']
Medoid is document with id 4957
Cluster is composed by 7 documents
documents in cluster -> [4949, 4951, 4956, 4957, 4958, 4961, 4963]

Cluster 2:
Centroid has top words ['service', 'order', 'begin', 'want', 'number']
Medoid is document with id 4953
Cluster is composed by 3 documents
documents in cluster -> [4950, 4953, 4954]

Cluster 3:
Centroid has top words ['high', 'day', '10', 'say', 'market']
Medoid is document with id 4948
Cluster is composed by 6 documents
documents in cluster -> [4948, 4952, 4955, 4959, 4960, 4962]

Example Execution with topics

The clustering solution has k = 4 clusters

Cluster 1:
Centroid has top words ['payment', 'balance', 'civil', 'accident', 'institution']
Medoid is topic with id 145
Cluster is composed by 6 topics
topics in cluster -> [145, 146, 147, 149, 150, 155]

Cluster 2:
Centroid has top words

# 3.2 -  Supervised approach: incorporating relevance feedback

**training(q, d_train, r_train, kwargs)** - Trains a classification model for a given topic

**Input:** (q, d_train, r_train, kwargs)

         q - topic document
         d_train - training collection
         r_train - judgements
         kwargs - Optional parameters with the following functionality (default values prefixed by *)
            classifier [*multinomialnb | kneighbors | randomforest | mlp] - Classifier model used for training
    
**Behaviour:** Learns a classification model to predict the relevance of documents on the topic q using the documents in the d_train collection, and their respective judgements r_train, that have been labeled for topic q. The vectors created from he subset of d_train relevant for topic q are stored in list for use in the classification function.

**Output:** q-conditional classification model

In [45]:
def training(q, d_train, r_train, **kwargs):
    global topic_vectorizers

    classifiers = {'multinomialnb': MultinomialNB(), 'kneighbors': KNeighborsClassifier(), 'randomforest': RandomForestClassifier(), 'mlp': MLPClassifier()}
    classifier = classifiers['multinomialnb'] if 'classifier' not in kwargs else classifiers[kwargs['classifier']]

    r_labels = find_R_test_labels(r_train[q])

    subset_dtrain = {}
    for doc in r_labels:
        subset_dtrain[doc] = d_train[doc]

    vec_results = create_vectorizer(subset_dtrain, 'tf-idf', **kwargs)
    topic_vectorizers[q] = vec_results[0]
    d_train_vec = vec_results[1]
    
    r_labels = list(r_labels.values())
    
    classifier.fit(X=d_train_vec, y=r_labels)

    return classifier


# Some simple example executions
#D_train = get_files_from_directory('../rcv1/', None, set='train')[1]
#D_train = process_collection(D_train, None)
D_train = read_from_file('collections_processed/Dtrain_judged_collection_processed')

r_set = get_R_set(material_dic)[0]
r_train = r_set[1]

Q_Test = [120,140,160,180]
classifiers = ['multinomialnb', 'kneighbors', 'randomforest', 'mlp']
example_models = []

for i in range(len(Q_Test)):
    print('\nTraining with q = {}'.format(Q_Test[i]))
    example_models.append(training(Q_Test[i], D_train, r_train, classifier=classifiers[i]))
    print(example_models[-1])


Training with q = 120
MultinomialNB()

Training with q = 140
KNeighborsClassifier()

Training with q = 160
RandomForestClassifier()

Training with q = 180
MLPClassifier()


**classify(d,q,M,kwargs)** - Calculates the probability of a document being relevant for a given topic
 
**Input:** (d,q,M,kwargs)
            
           d - document
           q - topic
           M - classification model

**Behaviour:** classifies the probability of document d to be relevant for topic q given M, using a vectorizer created during the training function

**Output:** probabilistic classification output on the relevance of document d to the topic q

In [56]:
def classify(d, q, M, **kwargs):
    vec = None
    vectorizers = topic_vectorizers[q]

    if type(vectorizers) != list:
        vec = vectorizers.transform(d)
    else:
        vec = vectorizers[1].transform(vectorizers[0].transform(d))

    return M.predict_proba(vec)[0][1]


# Some simple example executions
example_docs = get_files_from_directory('../rcv1_test/19960821/', None)[1][0:4]
example_docs = list(process_collection(example_docs, False).values())

for i in range(len(Q_Test)):
    print('\nExample execution for topic {}'.format(Q_Test[i]))
    print('Probability of being relevant for topic is {}%'.format(classify([example_docs[i]], Q_Test[i], example_models[i])*100))


Example execution for topic 120
Probability of being relevant for topic is 12.121054825591909%

Example execution for topic 140
Probability of being relevant for topic is 20.0%

Example execution for topic 160
Probability of being relevant for topic is 4.0%

Example execution for topic 180
Probability of being relevant for topic is 0.6774984037928257%


**evaluate(q_test, d_test, r_test, kwargs)** - Evaluates accuracy, precision and recall of the IR system using relevance feedback
 
**Input:**  (q_test, d_test, r_test, kwargs)

            q_test - subset of topics
            d_test - testing document collection
            r_test - judgements
            kwargs - Optional parameters with the following functionality (default values prefixed by *)
               ranking [*False | True] - Use of page ranking in evaluation
               p [*5 | int] - Number of pages displayed in page ranking
 
**Behaviour:** evaluates the behavior of the IR system in the presence of relevance feedback. Training and testing functions are called for each topic in Qtest for a more comprehensive assessment. Prints performance statistics regarding the underlying classification system and the behavior of the aided IR system

**Output:** Returns total accuracy of the IR system

In [38]:
def evaluate(q_test, d_test, r_test, **kwargs):
    ranking = False if 'ranking' not in kwargs else kwargs['ranking']
    p = 5 if 'top_p' not in kwargs else kwargs['top_p']

    total_accuracy = 0

    for q in q_test:

        sol_labels = []
        o_labels_training = []
        trained_probs = {}

        trained_classifier = training(q, d_train, r_train, **kwargs)

        judged_docs = []
        for doc_id in r_test[q]:
            judged_docs.append(doc_id)
        
        for doc_id in judged_docs:

            trained_prob = classify([d_test[doc_id]], q, trained_classifier)
            trained_probs[doc_id] = trained_prob
            sol_labels.append(r_test[q][doc_id])
            o_labels_training.append(1 if trained_prob > 0.5 else 0)

        results = {}
        
        results['accuracy'] = accuracy_score(sol_labels, o_labels_training)
        results['precision-micro'] = precision_score(sol_labels, o_labels_training, average='micro', zero_division=1)
        results['precision-macro'] = precision_score(sol_labels, o_labels_training, average='macro', zero_division=1)
        results['recall-micro'] =  recall_score(sol_labels, o_labels_training, average='micro')
        results['recall-macro'] =  recall_score(sol_labels, o_labels_training, average='macro')
        results['f-beta-micro'] = fbeta_score(sol_labels, o_labels_training, average='micro', beta=0.5)
        results['f-beta-macro'] = fbeta_score(sol_labels, o_labels_training, average='macro', beta=0.5)

        if ranking:
            sort_probs = sorted(trained_probs, key = trained_probs.get, reverse=True)

            ranked_result = []
            result_range = range(p) if p <= len(sort_probs) else range(len(sort_probs))
            for i in result_range:
                doc_id = sort_probs[i]
                ranked_result.append((doc_id, round(trained_probs[doc_id], 4)))

            results['ranking'] = ranked_result

        display_results(q, results)

        total_accuracy += results['accuracy']

    return total_accuracy / len(q_test)

# 3.3 - Graph ranking approach: document centrality

**build_graph(D, sim, theta, kwargs)** - Builds a document graph from document collection D using
the similarity measure in sim agains theta threshold

**Input:** (D, sim, theta, kwargs)

            D - The document collection to build our graph with
            sim - [*cosine | eucledian | manhattan] : The similarity measure used
            theta - The similarity threshold 

**Behaviour:** This function starts by creating the necessary structures for each of the give graph entries, and then proceeds to calculate the necessary pairwise similarity measures. It does so by treating each individual document as the query document and comparing it to all the rest.

**Output:** A dictionary representing the weighted undirect graph that connect all documents on the basis of the given similarity measure

In [39]:
def build_graph(D, sim, theta, **kwargs):
    doc_dic = process_collection(D, False, **kwargs)

    tfidf_vectorizer_info = tfidf_process(doc_dic, **kwargs)

    vectorizer = tfidf_vectorizer_info[0]
    doc_keys = tfidf_vectorizer_info[1]
    doc_vectors = tfidf_vectorizer_info[2]

    graph = {}
    for doc in doc_dic:
        graph[doc] = {}

    sim_method = sim_method_helper(sim)

    for doc in doc_dic:
        similarity_dic = sim_method([doc_dic[doc]], vectorizer, doc_keys, doc_vectors, theta, **kwargs)

        for simil_doc in similarity_dic:
            if doc != simil_doc:
                graph[doc][simil_doc] = similarity_dic[simil_doc]
                graph[simil_doc][doc] = similarity_dic[simil_doc]

    return graph

**undirected_page_rank(q, D, p, sim, theta, kwargs)** - This function applies a modified version of the PageRank algorithm for undirected graphs to the provided document collection, retriving the top p documents for topic q in regars to similarity measure sim and treshold theta. 

**Input:** (q, D, p, sim, theta, kwargs)
        
        q - A topic query in the form of topic identifier (int)
        D - The document collection we built our graph with
        p - The number of top documents to return
        sim - [*cosine | eucledian | manhattan] : The similarity measure used
        theta - The similarity threshold
        kwargs - Optional parameters with the following functionality (default values prefixed by *)
               sim_weight [*0.5 | float in [0.0, 1.0] ]: The weight given to the base similarity measure
               over the PageRank results 
 
**Behaviour:** This function serves primarily as an encapsulation for the PageRank algorithm, and as such it starts by creating the necessary structures for it to run, namely the link_graph. Afterwards, it  takes the PageRank results present in PageRank and weights the final results in accordance to the  results from the similarity measure sim given similiraty weight sim_weight. In the end it selects the top p perfoming documents for query q and returns them in list form.

**Output:** A list of ordered top-documents with their corresponding score in the form (d, score), ordered in descending order of score.

In [40]:
def undirected_page_rank(q, D, p, sim, theta, **kwargs):
    link_graph = build_graph(D, sim, theta, **kwargs)
    ranked_graph = page_rank(link_graph, q, D, **kwargs)

    query = topics[q]
    sim_method = sim_method_helper(sim)

    tdidf_info = tfidf_process(process_collection(D, False), **kwargs)
    vectorizer = tdidf_info[0]
    doc_keys = tdidf_info[1]
    doc_vectors = tdidf_info[2]

    sim_dic = sim_method([query], vectorizer, doc_keys, doc_vectors, 0, **kwargs)

    sim_weight = 0.5 if 'sim_weight' not in kwargs else kwargs['sim_weight']
    pr_weight = 1 - sim_weight

    ranked_graph = normalize_dic(ranked_graph, norm_method='zscore')
    sim_dic = normalize_dic(sim_dic, norm_method='zscore')
    
    for doc in sim_dic:
        sim_dic[doc] = sim_weight * sim_dic[doc] + pr_weight * ranked_graph[doc]

    # Retrieve top p documents
    sorted_dic = sorted(sim_dic, key = sim_dic.get, reverse=True)

    result = []
    result_range = range(p) if p <= len(sorted_dic) else range(len(sorted_dic))
    for i in result_range:
        doc = sorted_dic[i]
        result += [(doc, sim_dic[doc]),]

    return result